Binomial Tree
---

In [10]:
from scipy.stats import norm
from scipy.stats import t
import numpy as np
import math

St = 50
r = 0.1
q = 0
sigma = 0.4
tao = 0.25
T = 0.5
N = 100
 
S_max = 50
n_simulations = 10000
n_steps = 20

dt = (T-tao)/N
u = np.exp( sigma * np.sqrt(dt))
d = np.exp( - sigma * np.sqrt(dt))
p = (np.exp((r - q) * dt) - d) / (u - d)  # risk-neutral probability

#建立空間
s_tree = np.zeros((N+1,N+1))
s_tree[0,0] = St
s_max_tree = np.zeros((N+1,N+1,2*N+1))
s_max_tree[0,0,0] = S_max
option_tree = np.zeros((N+1,N+1,2*N+1))
epsilon = 1e-10 #判斷標準

#price tree
for i in range(1,N+1):
    s_tree[i,0] = s_tree[i-1,0] * u
    for j in range(1,i+1):
        s_tree[i,j] = s_tree[i-1,j-1] * d


##max price tree
for t in range(1,N+1):
    for i in range(0,t+1):
        if i == 0:
            if s_tree[t,i] >= S_max:
                s_max_tree[t,i,0] = s_tree[t,i]
            else:
                s_max_tree[t,i,0] = s_max_tree[0,0,0]
        elif i != t:
            for j in range(2*N+1+1):
                if s_max_tree[t-1,i-1,j] != 0:
                    s_max_tree[t,i,j] = s_max_tree[t-1,i-1,j]
                else:
                    break
            for k in range(2*N+1):
                if s_max_tree[t-1,i,k] != 0  and s_max_tree[t-1,i,k] >= s_tree[t,i] and s_max_tree[t-1,i,k] not in s_max_tree[t,i]:
                    for l in range(2*N+1+1):
                        if s_max_tree[t,i,l] == 0:
                           s_max_tree[t,i,l] = s_max_tree[t-1,i,k] 
                           break
        else:
            s_max_tree[t,i,0] = S_max

## option value in T
for i in range(0,t + 1):
    for k in range(2 * N + 1):
            if s_max_tree[N, i, k] != 0:
                option_tree[N, i, k] = s_max_tree[N, i, k] - s_tree[N, i]
            else:
                break
            
def option_type(option = "American"):
    for t in range(N-1,-1,-1):
        for i in range(0, t+1):
            for k in range(2*N+1):
                if s_max_tree[t,i,k] != 0:
                    
                    idx = np.where(s_max_tree[t + 1, i] == s_max_tree[t, i, k])[0]
                    if len(idx) > 0:
                        idx = idx[0]
                        stu = option_tree[t + 1 , i  , idx]
                    else:
                        idx = np.where(s_max_tree[t + 1, i] == s_tree[t + 1, i])[0]
                        idx = idx[0]
                        stu = option_tree[t + 1, i , idx]

                    idx = np.where(s_max_tree[t + 1, i + 1] == s_max_tree[t, i, k])[0]
                    idx = idx[0]
                    std = option_tree[t + 1, i + 1, idx]

                    hold = np.exp(-r * dt) * (p * stu + (1 - p) * std)
                    exercise = s_max_tree[t, i, k] - s_tree[t,i]
                    if option == "American":
                        option_tree[t,i,k] = np.maximum(hold,exercise)
                    elif option == "European":
                        option_tree[t,i,k] = hold

    return option_tree[0,0,0]

American = option_type(option = "American")
European = option_type(option = "European")

print(f"American_option: {round(American,4)}")
print(f"European_option: {round(European,4)}")

American_option: 7.4396
European_option: 7.2369


Monte_carlo
---

In [11]:
def MonteCarlo(S0,S_max,r,T,q,sigma,N,n_simulations,n_steps):
    dt=T/N
    #length of time step
    put_prices = []
    for _ in range(n_steps):
    #create two dimensional matrix of random std normal numbers (paths X timesteps)
        rn=np.random.standard_normal((N+1,n_simulations))
        #initialize S[0] to 100 stock price
        S=np.zeros_like(rn)
        S[0]=S0 
    
        for t in range(1,N+1): 
            S[t]=S[t-1]*np.exp((r-q-sigma**2/2)*dt+(sigma*rn[t]*math.sqrt(dt)))
        PayoffLookFloatPut=np.zeros(n_simulations)
        for i in range(0,n_simulations):
            S_max_path = np.max(S[:, i])
            if S_max > S_max_path:
                PayoffLookFloatPut[i] = np.maximum(S_max - S[-1, i], 0)
            else:
                PayoffLookFloatPut[i]=np.maximum(max(S[:,i])-S[-1,i],0)
        put_prices.append(math.exp(-r*(T))*PayoffLookFloatPut.mean())
    mean = np.mean(put_prices)
    std = np.std(put_prices)
    upper = np.round(mean + 2 * std,4)
    lower = np.round(mean - 2 * std,4)      
    CI_put = [lower, upper]    
    #Now Discount the Average of all payoffs
    return mean,std,CI_put 


mean_put,std_put,CI_put = MonteCarlo(St, S_max, r, T-tao, q, sigma,N, n_simulations, n_steps)
print(f"mean of put: {mean_put:.4f}", f"std of call: {std_put:.4f}", f"CI of call: {CI_put}")

mean of put: 7.1580 std of call: 0.0475 CI of call: [7.063, 7.2529]


Bonus 1

In [12]:
s_tree = np.zeros((N+1,N+1))
for t in range(N+1):
    for i in range(t+1):
        s_tree[t,i] = St*(u**(t-i))*(d**i)

option_tree = np.zeros((N+1,N+1,N+1))    
s_max_tree = np.zeros((N+1,N+1,N+1))
s_max_tree[0,0,0] = S_max

for t in range(N+1):
    for i in range(t+1):
        a = t-i
        if a >= i:
            for j in range(i+1):
                if St*(u**(a-j)) >= S_max:
                    s_max_tree[t,i,j] = St*(u**(a-j))
                else:
                    s_max_tree[t,i,j] = S_max
                    break

        else:
            for j in range(a+1):
                if St*(u**(a-j)) >= S_max:
                    s_max_tree[t,i,j] = St*(u**(a-j))
                else:
                    s_max_tree[t,i,j] = S_max
                    break


for i in range(N+1):
    for k in range(N+1):
        if s_max_tree[N,i,k] != 0:
            option_tree[N,i,k] = s_max_tree[N,i,k] - s_tree[N,i]
        else:
            break


for t in range(N-1,-1,-1):
    for i in range(0, t+1):
        for k in range(N+1):
            if s_max_tree[t,i,k] != 0:
                
                idx = np.where(s_max_tree[t+1,i] == s_max_tree[t,i,k])[0]
                if len(idx) > 0:
                    idx = idx[0]
                    stu = option_tree[t+1,i,idx]
                else:
                    a = s_tree[t+1,i]
                    b = s_max_tree[t+1,i]
                    idx = np.where(b-a < epsilon)[0]
                    idx = idx[0]
                    stu = option_tree[t+1,i,idx]

                idx = np.where(s_max_tree[t+1,i+1] == s_max_tree[t,i,k])[0]
                idx = idx[0]
                std = option_tree[t+1,i+1,idx]

                hold = np.exp(-r*dt) * (p*stu+(1-p)*std)
                exercise = s_max_tree[t,i,k]-s_tree[t,i]
                option_tree[t,i,k] = hold



print(f"European_option: {round(option_tree[0,0,0],4)}")

European_option: 7.2369
